# Machine Learning


Agora, após fazer uma análise exploratória dos dados e transformá-los conforme necessidade, podemos ir para os modelos de classificação.
Vamos trabalhar com três: RandomForest, XGBoost, LightGBM

In [ ]:
# 1. Converte de Spark para Pandas
df_pd = df_raw.toPandas()
df_pd_test = df_test.toPandas()

# 2. Trata strings que viram NaN no Pandas
for df in [df_pd, df_pd_test]:
    df.replace(["N/A", "NM", "na", "NaN", "-", ""], pd.NA, inplace=True)

# 3. Converte colunas para números (menos ID e target)
for col in df_pd.columns:
    if col not in ["ID", "Customer_ID", "Month", "Credit_Score"]:
        df_pd[col] = pd.to_numeric(df_pd[col], errors='coerce')
        if col in df_pd_test.columns:
            df_pd_test[col] = pd.to_numeric(df_pd_test[col], errors='coerce')

# 4. Remove nulos no target e preenche o resto com zero
df_pd = df_pd.dropna(subset=["Credit_Score"])
# Fill numerical columns with 0, leave 'Credit_Score' as is for encoding
numerical_cols_to_fill = df_pd.select_dtypes(include=["number"]).columns.tolist()
df_pd[numerical_cols_to_fill] = df_pd[numerical_cols_to_fill].fillna(0)

numerical_cols_test_to_fill = df_pd_test.select_dtypes(include=["number"]).columns.tolist()
df_pd_test[numerical_cols_test_to_fill] = df_pd_test[numerical_cols_test_to_fill].fillna(0)


# 5. Seleciona colunas numéricas válidas e o target
X = df_pd.drop(columns=["Credit_Score", "ID", "Customer_ID"])
X = X.select_dtypes(include=["number"])
y = df_pd["Credit_Score"] # Keep target as is for now

X_external = df_pd_test.drop(columns=["ID", "Customer_ID", "Credit_Score"], errors="ignore") # Drop Credit_Score from test if it exists
X_external = X_external.select_dtypes(include=["number"])

# --- Start of added code ---
# Encode the target variable
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
# --- End of added code ---

# 6. Split e escalonamento (using encoded y)
# Use y_encoded for the split
X_train, X_val, y_train_encoded, y_val_encoded = train_test_split(X, y_encoded, test_size=0.2, stratify=y_encoded, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_external_scaled = scaler.transform(X_external)

# 7. Modelos e seus grids (no change needed here)
models = {
    "RandomForest": {
        "model": RandomForestClassifier(random_state=42),
        "params": {
            "n_estimators": [100, 200],
            "max_depth": [None, 10],
            "min_samples_split": [2, 5]
        }
    },
    "XGBoost": {
        # use_label_encoder is deprecated, eval_metric='logloss' is good
        "model": XGBClassifier(eval_metric='logloss', random_state=42),
        "params": {
            "n_estimators": [100, 200],
            "max_depth": [3, 6],
            "learning_rate": [0.1, 0.3]
        }
    },
    "LightGBM": {
        "model": LGBMClassifier(random_state=42),
        "params": {
            "n_estimators": [100, 200],
            "num_leaves": [31, 50],
            "learning_rate": [0.1, 0.3]
        }
    }
}

# 8. GridSearch, avaliação e predição externa (using encoded y)
for name, config in models.items():
    print(f"\nTreinando {name}...")

    # Cronômetro inicia
    start_time = time.time()

    # Cronômetro inicia
    gs = GridSearchCV(config["model"], config["params"], cv=3, scoring="accuracy", n_jobs=-1)
    # Fit using the encoded training labels
    gs.fit(X_train_scaled, y_train_encoded)

    end_time = time.time()
    duration = end_time - start_time
    print(f"Tempo de treinamento para {name}: {duration:.2f} segundos")

    print("Melhores parâmetros:", gs.best_params_)

    # Predict and evaluate using the encoded validation labels
    y_val_pred_encoded = gs.best_estimator_.predict(X_val_scaled)
    # Decode predictions back to original labels for classification report if needed, or keep encoded
    # For classification_report, you can use the encoded labels and target_names
    target_names = label_encoder.classes_ # Get original class names
    print("Relatório de classificação (validação):")
    print(classification_report(y_val_encoded, y_val_pred_encoded, target_names=target_names))


    # Teste externo
    print("Predições no conjunto externo:")
    y_ext_pred_encoded = gs.best_estimator_.predict(X_external_scaled)
    # Decode external predictions if you need the original labels
    y_ext_pred = label_encoder.inverse_transform(y_ext_pred_encoded)
    print(pd.Series(y_ext_pred).value_counts())

    # Avalia se o df_test possui o target real
    if "Credit_Score" in df_pd_test.columns:
        y_ext_real = df_pd_test["Credit_Score"]
        # Encode the real external target before comparing
        y_ext_real_encoded = label_encoder.transform(y_ext_real)
        print("Relatório de classificação (dados externos):")
        # Use encoded predictions and encoded real values for classification report
        print(classification_report(y_ext_real_encoded, y_ext_pred_encoded, target_names=target_names))


Treinando RandomForest...
Tempo de treinamento para RandomForest: 509.65 segundos
Melhores parâmetros: {'max_depth': None, 'min_samples_split': 2, 'n_estimators': 200}
Relatório de classificação (validação):
              precision    recall  f1-score   support

        Good       0.77      0.69      0.73      3566
        Poor       0.79      0.80      0.80      5799
    Standard       0.80      0.82      0.81     10635

    accuracy                           0.79     20000
   macro avg       0.79      0.77      0.78     20000
weighted avg       0.79      0.79      0.79     20000

Predições no conjunto externo:
Standard    27711
Poor        14393
Good         7896
Name: count, dtype: int64

Treinando XGBoost...
Tempo de treinamento para XGBoost: 97.26 segundos
Melhores parâmetros: {'learning_rate': 0.3, 'max_depth': 6, 'n_estimators': 200}
Relatório de classificação (validação):
              precision    recall  f1-score   support

        Good       0.72      0.63      0.67      35

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.006861 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1982
[LightGBM] [Info] Number of data points in the train set: 80000, number of used features: 15
[LightGBM] [Info] Start training from score -1.724428
[LightGBM] [Info] Start training from score -1.237917
[LightGBM] [Info] Start training from score -0.631605
Tempo de treinamento para LightGBM: 88.25 segundos
Melhores parâmetros: {'learning_rate': 0.3, 'n_estimators': 200, 'num_leaves': 50}


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


Relatório de classificação (validação):
              precision    recall  f1-score   support

        Good       0.74      0.65      0.69      3566
        Poor       0.78      0.75      0.76      5799
    Standard       0.77      0.81      0.79     10635

    accuracy                           0.77     20000
   macro avg       0.76      0.74      0.75     20000
weighted avg       0.77      0.77      0.77     20000

Predições no conjunto externo:


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


Standard    28073
Poor        13885
Good         8042
Name: count, dtype: int64


Ao fazer uma análise do uso de cada modelo, temos que:

Random Forest
*   Accuracy: 0.79
*   Melhor performance geral (acurácia)

XGBoost:
*   Accuracy: 0.75

LightGBM:
*   Accuracy: 0.77

Se formos olhar para o recall, temos os seguintes resultados para cada um:

Random Forest:
*   Good: 0.69
*   Poor: 0.80
*   Standard: 0.82

XGBoost:
*   Good: 0.63
*   Poor: 0.73
*   Standard: 0.81

LightGBM:
*   Good: 0.65
*   Poor: 0.75
*   Standard: 0.81

Random Forest foi o mais equilibrado entre as três classes, seguido por LightGBM e XGBoost.

O tempo de processamento foi bem diferente, sendo o de random forest quase 6x o dos outros dois (9 minutos x 1 minuto e meio).

Um recall mais baixo para "Good" já era esperado, visto que temos a base desbalanceada, com:

 Credit_Score:
 *   Standard: 53174 (53.2%)
 *   Poor: 28998 (29.0%)
 *   Good: 17828 (17.8%)

Podemos como próximo passo aplicar um Random Forest forçando o algoritmo a compensar o desbalanceamento.

In [ ]:
# 1. Converte de Spark para Pandas
df_pd = df_raw.toPandas()
df_pd_test = df_test.toPandas()

# 2. Trata strings que viram NaN no Pandas
for df in [df_pd, df_pd_test]:
    df.replace(["N/A", "NM", "na", "NaN", "-", ""], pd.NA, inplace=True)

# 3. Converte colunas para números (menos ID e target)
for col in df_pd.columns:
    if col not in ["ID", "Customer_ID", "Month", "Credit_Score"]:
        df_pd[col] = pd.to_numeric(df_pd[col], errors='coerce')
        if col in df_pd_test.columns:
            df_pd_test[col] = pd.to_numeric(df_pd_test[col], errors='coerce')

# 4. Remove nulos no target e preenche o resto com zero
df_pd = df_pd.dropna(subset=["Credit_Score"])
# Fill numerical columns with 0, leave 'Credit_Score' as is for encoding
numerical_cols_to_fill = df_pd.select_dtypes(include=["number"]).columns.tolist()
df_pd[numerical_cols_to_fill] = df_pd[numerical_cols_to_fill].fillna(0)

numerical_cols_test_to_fill = df_pd_test.select_dtypes(include=["number"]).columns.tolist()
df_pd_test[numerical_cols_test_to_fill] = df_pd_test[numerical_cols_test_to_fill].fillna(0)


# 5. Seleciona colunas numéricas válidas e o target
X = df_pd.drop(columns=["Credit_Score", "ID", "Customer_ID"])
X = X.select_dtypes(include=["number"])
y = df_pd["Credit_Score"] # Keep target as is for now

X_external = df_pd_test.drop(columns=["ID", "Customer_ID", "Credit_Score"], errors="ignore") # Drop Credit_Score from test if it exists
X_external = X_external.select_dtypes(include=["number"])

# --- Start of added code ---
# Encode the target variable
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
# --- End of added code ---

# 6. Split e escalonamento (using encoded y)
# Use y_encoded for the split
X_train, X_val, y_train_encoded, y_val_encoded = train_test_split(X, y_encoded, test_size=0.2, stratify=y_encoded, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_external_scaled = scaler.transform(X_external)

# 7. Modelos e seus grids (no change needed here)
models = {
    "RandomForest": {
        "model": RandomForestClassifier(class_weight='balanced', random_state=42),
        "params": {
            "n_estimators": [100, 200],
            "max_depth": [None, 10],
            "min_samples_split": [2, 5]
        }
    }
}

# 8. GridSearch, avaliação e predição externa (using encoded y)
for name, config in models.items():
    print(f"\nTreinando {name}...")
    # Cronômetro inicia
    start_time = time.time()

    gs = GridSearchCV(config["model"], config["params"], cv=3, scoring="accuracy", n_jobs=-1)
    # Fit using the encoded training labels
    gs.fit(X_train_scaled, y_train_encoded)

    end_time = time.time()
    duration = end_time - start_time
    print(f"Tempo de treinamento para {name}: {duration:.2f} segundos")

    print("Melhores parâmetros:", gs.best_params_)

    # Predict and evaluate using the encoded validation labels
    y_val_pred_encoded = gs.best_estimator_.predict(X_val_scaled)
    # Decode predictions back to original labels for classification report if needed, or keep encoded
    # For classification_report, you can use the encoded labels and target_names
    target_names = label_encoder.classes_ # Get original class names
    print("Relatório de classificação (validação):")
    print(classification_report(y_val_encoded, y_val_pred_encoded, target_names=target_names))


    # Teste externo
    print("Predições no conjunto externo:")
    y_ext_pred_encoded = gs.best_estimator_.predict(X_external_scaled)
    # Decode external predictions if you need the original labels
    y_ext_pred = label_encoder.inverse_transform(y_ext_pred_encoded)
    print(pd.Series(y_ext_pred).value_counts())

    # Avalia se o df_test possui o target real
    if "Credit_Score" in df_pd_test.columns:
        y_ext_real = df_pd_test["Credit_Score"]
        # Encode the real external target before comparing
        y_ext_real_encoded = label_encoder.transform(y_ext_real)
        print("Relatório de classificação (dados externos):")
        # Use encoded predictions and encoded real values for classification report
        print(classification_report(y_ext_real_encoded, y_ext_pred_encoded, target_names=target_names))


Treinando RandomForest...


/usr/local/lib/python3.11/dist-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Tempo de treinamento para RandomForest: 506.93 segundos
Melhores parâmetros: {'max_depth': None, 'min_samples_split': 5, 'n_estimators': 200}
Relatório de classificação (validação):
              precision    recall  f1-score   support

        Good       0.75      0.74      0.75      3566
        Poor       0.78      0.82      0.80      5799
    Standard       0.81      0.80      0.81     10635

    accuracy                           0.79     20000
   macro avg       0.78      0.79      0.78     20000
weighted avg       0.79      0.79      0.79     20000

Predições no conjunto externo:
Standard    26609
Poor        14778
Good         8613
Name: count, dtype: int64


**Aplicação prática**:

O modelo pode ser utilizado pela QuantumFinance para:

*   Avaliar risco de crédito de novos clientes com base em histórico e comportamento financeiro
*   Segmentar clientes por perfil de risco (bom, padrão, ruim)
*   Apoiar decisões de concessão de crédito, limite de cartão ou taxas de juros personalizadas
*   Antecipar inadimplência, otimizando ações preventivas de cobrança